In [ ]:
from astroquery.esa.euclid import Euclid
import pandas as pd

In [ ]:
# Functions to query the schema
def list_tables():
    tables = Euclid.load_tables(only_names=True)
    for table in tables:
        print(table.name)


def get_schema(table_name):
    table = Euclid.load_table(table_name)
    print(f"Loaded {table.name} with {len(table.columns)} columns\n")

    data = []
    for col in table.columns:
        data.append([col.name, col.unit, col.description])

    df = pd.DataFrame(
        data,
        columns=["name", "unit", "description"],
    )

    return df.style.set_properties(**{"text-align": "left"}).set_table_styles(
        [{"selector": "th", "props": [("text-align", "left")]}]
    )

In [ ]:
list_tables()

In [ ]:
get_schema("catalogue.mer_catalogue")

In [ ]:
get_schema("catalogue.phz_classification")

In [ ]:
get_schema("catalogue.phz_photo_z")

In [ ]:
# Write query for Euclid data in ECDFS

# Cone centered on DP1 ECDFS
c_ra = 53.13
c_dec = -28.10
r = 0.7

query = f"""
SELECT
    m.object_id as euclid_id,
    m.right_ascension,
    m.declination,
    pz.flux_vis_unif,
    pz.flux_y_unif,
    pz.flux_j_unif,
    pz.flux_h_unif,
    pz.fluxerr_vis_unif,
    pz.fluxerr_y_unif,
    pz.fluxerr_j_unif,
    pz.fluxerr_h_unif
FROM
    catalogue.mer_catalogue as m
JOIN catalogue.phz_photo_z AS pz USING (object_id)
JOIN catalogue.phz_classification AS pc USING (object_id)
WHERE
    DISTANCE({c_ra}, {c_dec}, m.right_ascension, m.declination) < {r}
    AND m.spurious_flag = 0
    AND m.det_quality_flag < 4
    AND pc.phz_classification = 2
    AND m.vis_det = 1
    AND pz.phz_flags = 0
"""

In [ ]:
job = Euclid.launch_job_async(
    query,
    dump_to_file=True,
    output_file="../data/euclid_ecdfs.fits",
    output_format="fits",
    verbose=False,
)